<a href="https://colab.research.google.com/github/316104127/sistema_recomendacao_filmes/blob/main/sistemaderecomendacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly --upgrade
!pip install -U scikit-learn

In [10]:
#Criação dos caminhos para os datasets
path = '/content/Listagem_dos_Filmes_Brasileiros_e_Estrangeiros_Exibidos_2009_a_2019.csv'
path2 = '/content/IMDb movies.csv'

#Importações
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

## Carregamento e exibição dos datasets

In [ ]:
data_filme = pd.read_csv(path)
data_imdb = pd.read_csv(path2)

Dataset nacional.

In [4]:
data_filme.head(10)

,Unnamed: 0,Ano de exibição,Título da obra,Gênero,País(es) produtor(es) da obra,Nacionalidade da obra,Empresa distribuidora,Origem da empresa distribuidora,Público no ano de exibição,Renda (R$) no ano de exibição
0,0,2019,120 Batimentos Por Minuto,Ficção,França,Estrangeira,Imovision,Distribuição Nacional,10.0,70.0
1,1,2019,3 Faces,Ficção,Irã,Estrangeira,Imovision,Distribuição Nacional,6289.0,103805.0
2,2,2019,45 Dias Sem Você,Ficção,Brasil,Brasileira,O2 Play,Distribuição Nacional,3303.0,31295.0
3,3,2019,A 100 Passos De Um Sonho,Ficção,Estados Unidos,Estrangeira,Disney,Distribuição Internacional,69.0,4186.0
4,4,2019,A Abelhinha Maya - O Filme,Animação,Alemanha,Estrangeira,Playarte,Distribuição Nacional,402.0,3531.0
5,5,2019,A Batalha Das Correntes,Ficção,Estados Unidos,Estrangeira,Diamond Films,Distribuição Nacional,19579.0,425304.0
6,6,2019,A Bela da Tarde,Ficção,"França, Itália",Estrangeira,Zeta Filmes,Distribuição Nacional,14.0,253.0
7,7,2019,A Cabeça De Gumercindo Saraiva,Ficção,Brasil,Brasileira,Elo Company,Distribuição Nacional,195.0,1830.0
8,8,2019,A Caminho De Casa,Ficção,Estados Unidos,Estrangeira,Sony,Distribuição Internacional,652641.0,9445705.0
9,9,2019,A Casa De Veraneio,Ficção,"França, Itália",Estrangeira,Imovision,Distribuição Nacional,8001.0,137759.0


Dataset estrangeiro.

In [5]:
data_imdb.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [6]:
data_filme.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7052 entries, 0 to 7051
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       7052 non-null   int64  
 1   Ano de exibição                  7052 non-null   int64  
 2   Título da obra                   7052 non-null   object 
 3   Gênero                           7052 non-null   object 
 4   País(es) produtor(es) da obra    7052 non-null   object 
 5   Nacionalidade da obra            7052 non-null   object 
 6   Empresa distribuidora            7052 non-null   object 
 7   Origem da empresa distribuidora  7052 non-null   object 
 8   Público no ano de exibição       7052 non-null   float64
 9   Renda (R$) no ano de exibição    7052 non-null   float64
dtypes: float64(2), int64(2), object(6)
memory usage: 551.1+ KB


In [7]:
data_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          85855 non-null  object 
 1   title                  85855 non-null  object 
 2   original_title         85855 non-null  object 
 3   year                   85855 non-null  object 
 4   date_published         85855 non-null  object 
 5   genre                  85855 non-null  object 
 6   duration               85855 non-null  int64  
 7   country                85791 non-null  object 
 8   language               85022 non-null  object 
 9   director               85768 non-null  object 
 10  writer                 84283 non-null  object 
 11  production_company     81400 non-null  object 
 12  actors                 85786 non-null  object 
 13  description            83740 non-null  object 
 14  avg_vote               85855 non-null  float64
 15  vo

Tratamenos e Divisões

Como o dataset estrangeiro possui títulos originais sei que o mesmo terá a escrita igual a do país de origem do filme, assim consigo trazer todos os filmes em comum dos datasets e terei informações de avaliação(estrangeiro) e renda(nacional)

In [8]:
#Aqui é feito uma junção dos datas baseado nos títulos originais.
data_imdb.rename(columns= {"original_title":"Título"}, inplace=True)
data_filme.rename(columns= {"Título da obra":"Título"}, inplace=True)
data_juncao = data_imdb.merge(data_filme, on='Título')
data_juncao.head()

,imdb_title_id,title,Título,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Unnamed: 0,Ano de exibição,Gênero,País(es) produtor(es) da obra,Nacionalidade da obra,Empresa distribuidora,Origem da empresa distribuidora,Público no ano de exibição,Renda (R$) no ano de exibição
0,tt0007338,Sherlock Holmes,Sherlock Holmes,1916,1916-05-15,Mystery,116,USA,English,Arthur Berthelet,"H.S. Sheldon, William Gillette",The Essanay Film Manufacturing Company,"William Gillette, Marjorie Kay, Ernest Maupain...",When a couple of swindlers hold young Alice Fa...,6.4,313,NaN,NaN,NaN,NaN,10.0,16.0,6406,2010,Ficção,Estados Unidos,Estrangeira,Warner,Distribuição Internacional,2472825.0,21996832.0
1,tt0013597,Sherlock Holmes,Sherlock Holmes,1922,1922-03-07,"Drama, Mystery",85,USA,English,Albert Parker,"Earle Browne, Arthur Conan Doyle",Goldwyn Pictures Corporation,"John Barrymore, Roland Young, Carol Dempster, ...",A young Sherlock Holmes seeks to bring down th...,5.8,551,NaN,$ 384770,NaN,NaN,18.0,15.0,6406,2010,Ficção,Estados Unidos,Estrangeira,Warner,Distribuição Internacional,2472825.0,21996832.0
2,tt0023460,Sherlock Holmes,Sherlock Holmes,1932,1932-11-06,Drama,68,USA,English,William K. Howard,"William Gillette, Bertram Millhauser",Fox Film Corporation,"Clive Brook, Miriam Jordan, Ernest Torrence, H...",The famous detective is pulled away from retir...,5.7,131,NaN,NaN,NaN,NaN,12.0,7.0,6406,2010,Ficção,Estados Unidos,Estrangeira,Warner,Distribuição Internacional,2472825.0,21996832.0
3,tt0988045,Sherlock Holmes,Sherlock Holmes,2009,2009-12-25,"Action, Adventure, Mystery",128,"USA, Germany, UK, Australia","English, French",Guy Ritchie,"Michael Robert Johnson, Anthony Peckham",Warner Bros.,"Robert Downey Jr., Jude Law, Rachel McAdams, M...",Detective Sherlock Holmes and his stalwart par...,7.6,574055,$ 90000000,$ 209028679,$ 524028679,57.0,698.0,419.0,6406,2010,Ficção,Estados Unidos,Estrangeira,Warner,Distribuição Internacional,2472825.0,21996832.0
4,tt0012249,Amleto,Hamlet,1921,1921-01-27,Drama,131,Germany,German,"Svend Gade, Heinz Schall","Erwin Gepard, Edward P. Vining",Art-Film GmbH,"Asta Nielsen, Paul Conradi, Mathilde Brandt, E...",Danish silent movie-star Asta Nielsen formed h...,7.0,382,NaN,NaN,NaN,NaN,5.0,8.0,3181,2015,Ficção,Brasil,Brasileira,Bela Filmes,Distribuição Nacional,307.0,3771.0


In [9]:
#criação do dataset de exibição
data_exib = data_juncao[["imdb_title_id", "title",	"Público no ano de exibição"]]
data_exib.rename(columns= {"Público no ano de exibição":"Exib"}, inplace=True)
data_exib

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,imdb_title_id,title,Exib
0,tt0007338,Sherlock Holmes,2472825.0
1,tt0013597,Sherlock Holmes,2472825.0
2,tt0023460,Sherlock Holmes,2472825.0
3,tt0988045,Sherlock Holmes,2472825.0
4,tt0012249,Amleto,307.0
...,...,...,...
1273,tt9207188,Os Parças 2,1305768.0
1274,tt9213932,Kardec,749256.0
1275,tt9214280,Eu Sou Mais Eu,126718.0
1276,tt9522080,Non conosci Papicha,669.0


In [11]:
#criação do dataset de avaliação
data_ava = data_juncao[["imdb_title_id", "title",	"avg_vote", "votes"]]
data_ava

,imdb_title_id,title,avg_vote,votes
0,tt0007338,Sherlock Holmes,6.4,313
1,tt0013597,Sherlock Holmes,5.8,551
2,tt0023460,Sherlock Holmes,5.7,131
3,tt0988045,Sherlock Holmes,7.6,574055
4,tt0012249,Amleto,7.0,382
...,...,...,...,...
1273,tt9207188,Os Parças 2,3.7,108
1274,tt9213932,Kardec,6.1,1128
1275,tt9214280,Eu Sou Mais Eu,2.1,351
1276,tt9522080,Non conosci Papicha,7.1,1422


Procuro filmes com pelo menos 10000 visualizações e avaliação 4

In [38]:
data_exib2 = data_exib.set_index('imdb_title_id')
x = data_exib2['Exib'] >= 10000
x

imdb_title_id
tt0007338     True
tt0013597     True
tt0023460     True
tt0988045     True
tt0012249    False
             ...  
tt9207188     True
tt9213932     True
tt9214280     True
tt9522080    False
tt9626278    False
Name: Exib, Length: 1278, dtype: bool

In [39]:
y = x[x].index
print(y.shape)

(504,)


In [40]:
y

Index(['tt0007338', 'tt0013597', 'tt0023460', 'tt0988045', 'tt0013556',
       'tt0070608', 'tt0102797', 'tt0955308', 'tt4532826', 'tt0015224',
       ...
       'tt8513302', 'tt8591986', 'tt8660600', 'tt8716800', 'tt8772262',
       'tt8837044', 'tt8900098', 'tt9207188', 'tt9213932', 'tt9214280'],
      dtype='object', name='imdb_title_id', length=504)

In [ ]:
filme_exib = data_exib[data_exib['imdb_title_id'].isin(y)]
filme_exib['Exib'] = filme_exib['Exib'].where(filme_exib.Exib >= 10000)
filme_exib.dropna(inplace=True)

In [42]:
print('Valor mínimo de exibição:')
filme_exib['Exib'].min()

Valor mínimo de exibição:


10031.0

In [43]:
data_ava2 = data_ava.set_index('imdb_title_id')

#Meu X é possui as informações de verdadeiro e falso sobre quem atende o meu critério de avg_vote >= 4
x = data_ava2['avg_vote'] >= 4
x

imdb_title_id
tt0007338     True
tt0013597     True
tt0023460     True
tt0988045     True
tt0012249     True
             ...  
tt9207188    False
tt9213932     True
tt9214280    False
tt9522080     True
tt9626278     True
Name: avg_vote, Length: 1278, dtype: bool

In [44]:
#Quantidade de id que atendem ao meu critério
y = x[x].index
print(y.shape)

(1245,)


In [45]:
y #Meu Y possui oos ids que atentem ao meu critério

Index(['tt0007338', 'tt0013597', 'tt0023460', 'tt0988045', 'tt0012249',
       'tt0040416', 'tt0058175', 'tt0064400', 'tt0099726', 'tt0116477',
       ...
       'tt8731138', 'tt8731138', 'tt8772262', 'tt8837044', 'tt8900098',
       'tt9159056', 'tt9159056', 'tt9213932', 'tt9522080', 'tt9626278'],
      dtype='object', name='imdb_title_id', length=1245)

In [46]:
#Atraveés do Y fazemos uma junção apenas das linhas que atendem o critério
filme_ava = data_ava[data_ava['imdb_title_id'].isin(y)]
filme_ava

,imdb_title_id,title,avg_vote,votes
0,tt0007338,Sherlock Holmes,6.4,313
1,tt0013597,Sherlock Holmes,5.8,551
2,tt0023460,Sherlock Holmes,5.7,131
3,tt0988045,Sherlock Holmes,7.6,574055
4,tt0012249,Amleto,7.0,382
...,...,...,...,...
1271,tt9159056,Rasga Coração,6.9,186
1272,tt9159056,Rasga Coração,6.9,186
1274,tt9213932,Kardec,6.1,1128
1276,tt9522080,Non conosci Papicha,7.1,1422


In [47]:
print('Valor mínimo de avaliação:')
filme_ava['avg_vote'].min()

Valor mínimo de avaliação:


4.0

In [48]:
#Junção dos datas de avaliação e visualização
filmes_ava_exib = filme_exib.merge(filme_ava, on='imdb_title_id')
filmes_ava_exib.drop_duplicates(['imdb_title_id'], inplace=True)
filmes_ava_exib

,imdb_title_id,title_x,Exib,title_y,avg_vote,votes
0,tt0007338,Sherlock Holmes,2472825.0,Sherlock Holmes,6.4,313
1,tt0013597,Sherlock Holmes,2472825.0,Sherlock Holmes,5.8,551
2,tt0023460,Sherlock Holmes,2472825.0,Sherlock Holmes,5.7,131
3,tt0988045,Sherlock Holmes,2472825.0,Sherlock Holmes,7.6,574055
4,tt0013556,Robin Hood,1543953.0,Robin Hood,7.2,1971
...,...,...,...,...,...,...
773,tt8716800,Cinderela Pop,476714.0,Cinderela Pop,4.4,584
774,tt8772262,Midsommar - Il villaggio dei dannati,66073.0,Midsommar - Il villaggio dei dannati,7.1,176389
775,tt8837044,O Candidato Honesto 2,580568.0,O Candidato Honesto 2,5.0,219
776,tt8900098,Deslembro,16500.0,Deslembro,7.2,282


In [49]:
filmes_piv_exib = filmes_ava_exib.pivot_table(columns='imdb_title_id', index='title_x', values='Exib')
filmes_piv_ava = filmes_ava_exib.pivot_table(columns='imdb_title_id', index='title_x', values='avg_vote')

#Faço um pivo no meu dataset para estar na melhor forma de analise do meu ML
#aonde está nulo troco para zero
filmes_piv_ava.fillna(0, inplace=True)
filmes_piv_exib.fillna(0, inplace=True)

In [50]:
filmes_piv_ava.head(25)

imdb_title_id,tt0007338,tt0013556,tt0013597,tt0015224,tt0018107,tt0021310,tt0023460,tt0026071,tt0026956,tt0031092,tt0033563,tt0039277,tt0040098,tt0045911,tt0046183,tt0048657,tt0050650,tt0051565,tt0053362,tt0061359,tt0067934,tt0070511,tt0070608,tt0076792,tt0076881,tt0077452,tt0077651,tt0078768,tt0080554,tt0080923,tt0081787,tt0082017,tt0082559,tt0083564,tt0084854,tt0085672,tt0090184,tt0090352,tt0093952,tt0097826,...,tt6145054,tt6157970,tt6206880,tt6266538,tt6343058,tt6390668,tt6400280,tt6411984,tt6544318,tt6682820,tt6692022,tt6805354,tt6852272,tt6853930,tt6902696,tt6987798,tt7165346,tt7293380,tt7467324,tt7491144,tt7702334,tt7744802,tt7867358,tt7882104,tt7958736,tt7986672,tt8079248,tt8311958,tt8359848,tt8513302,tt8591986,tt8660600,tt8716800,tt8772262,tt8837044,tt8900098,tt8941440,tt8983180,tt9213932,tt9351598
title_x,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Busca,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Caixa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Festa da Menina Morta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Grande Vitória,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A História da Eternidade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Mulher Invisível,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Mulher do meu Amigo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
filmes_piv_exib.head(25)

imdb_title_id,tt0007338,tt0013556,tt0013597,tt0015224,tt0018107,tt0021310,tt0023460,tt0026071,tt0026956,tt0031092,tt0033563,tt0039277,tt0040098,tt0045911,tt0046183,tt0048657,tt0050650,tt0051565,tt0053362,tt0061359,tt0067934,tt0070511,tt0070608,tt0076792,tt0076881,tt0077452,tt0077651,tt0078768,tt0080554,tt0080923,tt0081787,tt0082017,tt0082559,tt0083564,tt0084854,tt0085672,tt0090184,tt0090352,tt0093952,tt0097826,...,tt6145054,tt6157970,tt6206880,tt6266538,tt6343058,tt6390668,tt6400280,tt6411984,tt6544318,tt6682820,tt6692022,tt6805354,tt6852272,tt6853930,tt6902696,tt6987798,tt7165346,tt7293380,tt7467324,tt7491144,tt7702334,tt7744802,tt7867358,tt7882104,tt7958736,tt7986672,tt8079248,tt8311958,tt8359848,tt8513302,tt8591986,tt8660600,tt8716800,tt8772262,tt8837044,tt8900098,tt8941440,tt8983180,tt9213932,tt9351598
title_x,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Busca,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Caixa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Festa da Menina Morta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Grande Vitória,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A História da Eternidade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Mulher Invisível,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Mulher do meu Amigo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
#matriz esparsa
fm_sparse = csr_matrix(filmes_piv_exib)
fm_sparse1 = csr_matrix(filmes_piv_ava)

In [53]:
model = NearestNeighbors(algorithm='brute')
model.fit(fm_sparse)


NearestNeighbors(algorithm='brute')

Predições

In [56]:
id_do_filme = 50


distances, suggestions = model.kneighbors(filmes_piv_exib.iloc[id_do_filme, :].values.reshape(1,-1))


for i in range(len(suggestions)):
  print(f'Indicações:\n {filmes_piv_exib.index[suggestions[i]]}')

Indicações:
 Index(['Augustine', 'Violette', 'Sangue Azul', 'O Banquete', 'Las acacias'], dtype='object', name='title_x')
